<a href="https://colab.research.google.com/github/Kate5-7-2021/COVID-19-KG/blob/main/Copy_of_covid_19_kg_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Preliminaries

In [ ]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [ ]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [ ]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


# 2. Loading a Knowledge Graph dataset

In [ ]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import pandas as pd
import requests
import io

In [ ]:
# Downloading the csv file from your GitHub account
re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/new_triples_with_predefined_relations_pdf_June20_.csv').content
re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/new_triples_with_predefined_relations_pmc_June20_.csv').content
pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
dataset = pd.concat([pmc, pdf])
print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'subject', 'object', 'new_relation'], dtype='object')


,subject,new_relation,object
0,dysfunction,disease_species,child
1,et dysfunction,disease_species,child
2,et dysfunction,disease_species,child
3,rsv,disease_disease,virus
4,treatment,treat_procedure_species,child


In [ ]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (230330, 3)


In [ ]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    63192
gene_gene                          24172
disease_species                    22370
gene_disease                       14220
disease_gene                       13347
disease_treat_procedure            10184
treat_procedure_disease             8223
drug_disease                        7250
gene_treat_procedure                6142
treat_procedure_treat_procedure     5625
gene_species                        5298
drug_gene                           5270
drug_drug                           4613
treat_procedure_gene                4602
disease_symptom                     4354
treat_procedure_species             4263
disease_drug                        4044
drug_species                        3680
gene_drug                           3603
symptom_disease                     3411
symptom_symptom                     3266
drug_treat_procedure                3229
treat_procedure_drug                2578
symptom_species               

## 2.1 Create training, validation and test splits

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen
# get the validation set of size 500
test_train, X_valid = train_test_split_no_unseen(dataset.values, 500, seed=0)

# get the test set of size 1000 from the remaining triples
X_train, X_test = train_test_split_no_unseen(test_train, 1000, seed=0)

print('Total triples:', dataset.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

Total triples: (230330, 3)
Size of train: (228830, 3)
Size of valid: (500, 3)
Size of test: (1000, 3)


# 3. Model Training

In [ ]:
from ampligraph.latent_features import TransE

# Train a KGE model
model = TransE(k=300, 
               epochs=100, 
               eta=1, 
               loss='multiclass_nll', 
               initializer='xavier', initializer_params={'uniform': False},
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
               optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
               seed= 0, batches_count= 10, verbose=True)

model.fit(X_train)
# ----------------------
# Evaluate: 
# Filtered evaluation with ranking strategy assigning worst rank to break ties

from ampligraph.utils import save_model, restore_model
save_model(model, 'TransE.pkl')
model = restore_model('TransE.pkl')

# create the filter 
X_filter = np.concatenate([X_train, X_valid, X_test], 0)

# compute ranks
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

# ranks are computed per triple
print('Test set:', X_test.shape)
print('Size of ranks:', ranks.shape)

# Aggregate metrics show the aggregate performance of the model on the test set using a single number
display_aggregate_metrics(ranks)
# ----------------------

100%|██████████| 1000/1000 [00:05<00:00, 172.67it/s]


Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 266.1925
Mean Reciprocal Rank: 0.19770617707036262
Hits@1: 0.1245
Hits@10: 0.3455
Hits@100: 0.664


# 4. Knowledge Discovery

### Relations

In [ ]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    63192
gene_gene                          24172
disease_species                    22370
gene_disease                       14220
disease_gene                       13347
disease_treat_procedure            10184
treat_procedure_disease             8223
drug_disease                        7250
gene_treat_procedure                6142
treat_procedure_treat_procedure     5625
gene_species                        5298
drug_gene                           5270
drug_drug                           4613
treat_procedure_gene                4602
disease_symptom                     4354
treat_procedure_species             4263
disease_drug                        4044
drug_species                        3680
gene_drug                           3603
symptom_disease                     3411
symptom_symptom                     3266
drug_treat_procedure                3229
treat_procedure_drug                2578
symptom_species               

##  Question - Answering

``` 
    <head, relation, ?> 
    <head, ?,        tail>
    <?,    relation, tail>
```

### Query 4: what are the possible therapeutic procedure of covid-19??

In [ ]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=15, 
                             head='covid-19', 
                             relation='disease_treat_procedure', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.224977493286133 	 ['covid-19' 'disease_treat_procedure' 'extracted'] 
Score: -10.50957202911377 	 ['covid-19' 'disease_treat_procedure' 'detected'] 
Score: -10.69274616241455 	 ['covid-19' 'disease_treat_procedure' 'destruction'] 
Score: -10.793622970581055 	 ['covid-19' 'disease_treat_procedure' 'induction'] 
Score: -10.830591201782227 	 ['covid-19' 'disease_treat_procedure' 'adjustment'] 
Score: -10.852009773254395 	 ['covid-19' 'disease_treat_procedure' 'expression'] 
Score: -10.876237869262695 	 ['covid-19' 'disease_treat_procedure' 'diet'] 
Score: -10.881346702575684 	 ['covid-19' 'disease_treat_procedure' 'infiltration'] 
Score: -10.923327445983887 	 ['covid-19' 'disease_treat_procedure' 'reduction'] 
Score: -10.929838180541992 	 ['covid-19' 'disease_treat_procedure' 'isolation'] 
Score: -10.93027114868164 	 ['covid-19' 'disease_treat_procedure' 'change'] 
Score: -10.931217193603516 	 ['covid-19' 'disease_treat_procedure' 'release'] 
Score: -10.940086364746094 	 ['covi

In [ ]:
# Query: what are the possible therapeutic procedure of covid-19??

from ampligraph.discovery import query_topn

# restore the previously saved model to save time
model = restore_model('TransE.pkl')

triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -11.123162269592285 	 ['vaccination' 'treat_procedure_disease' 'covid-19'] 
Score: -11.154512405395508 	 ['administration' 'treat_procedure_disease' 'covid-19'] 
Score: -11.207574844360352 	 ['reduction' 'treat_procedure_disease' 'covid-19'] 
Score: -11.22590446472168 	 ['detection' 'treat_procedure_disease' 'covid-19'] 
Score: -11.244354248046875 	 ['treatment' 'treat_procedure_disease' 'covid-19'] 
Score: -11.297548294067383 	 ['adoptive transfer' 'treat_procedure_disease' 'covid-19'] 
Score: -11.30508804321289 	 ['inoculation' 'treat_procedure_disease' 'covid-19'] 
Score: -11.320907592773438 	 ['injection' 'treat_procedure_disease' 'covid-19'] 
Score: -11.324865341186523 	 ['coagulation' 'treat_procedure_disease' 'covid-19'] 
Score: -11.32553768157959 	 ['isolation' 'treat_procedure_disease' 'covid-19'] 
Score: -11.343774795532227 	 ['change' 'treat_procedure_disease' 'covid-19'] 
Score: -11.347764015197754 	 ['enlargement' 'treat_procedure_disease' 'covid-19'] 
Score: -11.36

### Query 2: What are related diseases of Covid-19

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=15, 
                             head='covid-19', 
                             relation='disease_disease', 
                             tail= None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -12.165374755859375 	 ['covid-19' 'disease_disease' 'covid-19'] 
Score: -13.526830673217773 	 ['covid-19' 'disease_disease' '2009'] 
Score: -13.63525390625 	 ['covid-19' 'disease_disease' 'severe'] 
Score: -13.637168884277344 	 ['covid-19' 'disease_disease' 'infectious'] 
Score: -13.657071113586426 	 ['covid-19' 'disease_disease' 'fatal'] 
Score: -13.709989547729492 	 ['covid-19' 'disease_disease' 'major'] 
Score: -13.716577529907227 	 ['covid-19' 'disease_disease' 'contagious'] 
Score: -13.841659545898438 	 ['covid-19' 'disease_disease' 'chronic'] 
Score: -13.939491271972656 	 ['covid-19' 'disease_disease' 'transmissible'] 
Score: -13.947660446166992 	 ['covid-19' 'disease_disease' 'acute respiratory syndrome coronavirus 2'] 
Score: -14.008952140808105 	 ['covid-19' 'disease_disease' 'childhood'] 
Score: -14.129085540771484 	 ['covid-19' 'disease_disease' 'lower'] 
Score: -14.135295867919922 	 ['covid-19' 'disease_disease' 'multi-organ failure'] 
Score: -14.260622024536133 	 ['

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='disease_disease', 
                             tail= 'covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -12.165374755859375 	 ['covid-19' 'disease_disease' 'covid-19'] 
Score: -14.124876976013184 	 ['he' 'disease_disease' 'covid-19'] 
Score: -14.441706657409668 	 ['coronaviruses' 'disease_disease' 'covid-19'] 
Score: -14.49870491027832 	 ['alzheimer' 'disease_disease' 'covid-19'] 
Score: -14.559700012207031 	 ['noroviruses' 'disease_disease' 'covid-19'] 
Score: -14.576507568359375 	 ['blepharitis' 'disease_disease' 'covid-19'] 
Score: -14.577310562133789 	 ['sars cov infection' 'disease_disease' 'covid-19'] 
Score: -14.580307960510254 	 ['rabies' 'disease_disease' 'covid-19'] 
Score: -14.585343360900879 	 ['pneumoniae' 'disease_disease' 'covid-19'] 
Score: -14.597208023071289 	 ['fungal infection' 'disease_disease' 'covid-19'] 
Score: -14.631743431091309 	 ['dad' 'disease_disease' 'covid-19'] 
Score: -14.651535034179688 	 ['covid' 'disease_disease' 'covid-19'] 
Score: -14.671496391296387 	 ['cholera' 'disease_disease' 'covid-19'] 
Score: -14.684667587280273 	 ['hbov' 'disease_dise

### Query 1: What are the genes/proteins related to Covid-19

In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=15, 
                             head='covid-19', 
                             relation='disease_gene', 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -11.309284210205078 	 ['covid-19' 'disease_gene' 'rare'] 
Score: -11.598053932189941 	 ['covid-19' 'disease_gene' 'hong kong'] 
Score: -11.716827392578125 	 ['covid-19' 'disease_gene' 'spring'] 
Score: -11.957058906555176 	 ['covid-19' 'disease_gene' 'lobe'] 
Score: -12.004956245422363 	 ['covid-19' 'disease_gene' 'airway'] 
Score: -12.094305992126465 	 ['covid-19' 'disease_gene' 'wild'] 
Score: -12.152280807495117 	 ['covid-19' 'disease_gene' 'damage'] 
Score: -12.169944763183594 	 ['covid-19' 'disease_gene' 'small'] 
Score: -12.174610137939453 	 ['covid-19' 'disease_gene' 'cell'] 
Score: -12.176697731018066 	 ['covid-19' 'disease_gene' 'end'] 
Score: -12.213685989379883 	 ['covid-19' 'disease_gene' 'past'] 
Score: -12.216846466064453 	 ['covid-19' 'disease_gene' 'impact'] 
Score: -12.230427742004395 	 ['covid-19' 'disease_gene' 'host'] 
Score: -12.266889572143555 	 ['covid-19' 'disease_gene' 'bite'] 
Score: -12.269514083862305 	 ['covid-19' 'disease_gene' 'genome'] 


In [ ]:
# Predict relation

triples, scores = query_topn(model, top_n=15, 
                             head= None, 
                             relation='gene_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -9.722570419311523 	 ['she' 'gene_disease' 'covid-19'] 
Score: -9.748113632202148 	 ['il' 'gene_disease' 'covid-19'] 
Score: -9.954036712646484 	 ['sex' 'gene_disease' 'covid-19'] 
Score: -9.964839935302734 	 ['pattern' 'gene_disease' 'covid-19'] 
Score: -10.002358436584473 	 ['impact' 'gene_disease' 'covid-19'] 
Score: -10.004868507385254 	 ['damage' 'gene_disease' 'covid-19'] 
Score: -10.018580436706543 	 ['ace2' 'gene_disease' 'covid-19'] 
Score: -10.036943435668945 	 ['antibody' 'gene_disease' 'covid-19'] 
Score: -10.03921890258789 	 ['protein' 'gene_disease' 'covid-19'] 
Score: -10.052689552307129 	 ['all' 'gene_disease' 'covid-19'] 
Score: -10.107821464538574 	 ['gene' 'gene_disease' 'covid-19'] 
Score: -10.111820220947266 	 ['csf' 'gene_disease' 'covid-19'] 
Score: -10.11377239227295 	 ['autophagy' 'gene_disease' 'covid-19'] 
Score: -10.126688003540039 	 ['mabs' 'gene_disease' 'covid-19'] 
Score: -10.143289566040039 	 ['hcov nl63' 'gene_disease' 'covid-19'] 


### Query 2: What species transmit the covid-19

In [ ]:
# Predict relation
triples, scores = query_topn(model, top_n=30, 
                             tail=None, 
                             relation='disease_species', 
                             head='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -11.374004364013672 	 ['covid-19' 'disease_species' 'child'] 
Score: -11.460870742797852 	 ['covid-19' 'disease_species' 'dog'] 
Score: -11.500686645507812 	 ['covid-19' 'disease_species' 'cattle'] 
Score: -11.516824722290039 	 ['covid-19' 'disease_species' 'patient'] 
Score: -11.598112106323242 	 ['covid-19' 'disease_species' 'rabbit'] 
Score: -11.600951194763184 	 ['covid-19' 'disease_species' 'specie'] 
Score: -11.63563346862793 	 ['covid-19' 'disease_species' 'woman'] 
Score: -11.6459379196167 	 ['covid-19' 'disease_species' 'horse'] 
Score: -11.65051555633545 	 ['covid-19' 'disease_species' 'piglet'] 
Score: -11.67470645904541 	 ['covid-19' 'disease_species' 'swine'] 
Score: -11.689424514770508 	 ['covid-19' 'disease_species' 'calf'] 
Score: -11.718807220458984 	 ['covid-19' 'disease_species' 'sheep'] 
Score: -11.732292175292969 	 ['covid-19' 'disease_species' 'people'] 
Score: -11.736963272094727 	 ['covid-19' 'disease_species' 'human'] 
Score: -11.74500560760498 	 ['covid

### Query 3: what are symptoms of the covid-19

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='disease_symptom', 
                             head='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.652080535888672 	 ['covid-19' 'disease_symptom' 'discharge'] 
Score: -10.707016944885254 	 ['covid-19' 'disease_symptom' 'discomfort'] 
Score: -10.726543426513672 	 ['covid-19' 'disease_symptom' 'severe diarrhea'] 
Score: -10.758989334106445 	 ['covid-19' 'disease_symptom' 'imbalance'] 
Score: -10.762752532958984 	 ['covid-19' 'disease_symptom' 'finding'] 
Score: -10.780302047729492 	 ['covid-19' 'disease_symptom' 'organ failure'] 
Score: -10.82988166809082 	 ['covid-19' 'disease_symptom' 'symptom'] 
Score: -10.914045333862305 	 ['covid-19' 'disease_symptom' 'insomnia'] 
Score: -10.963275909423828 	 ['covid-19' 'disease_symptom' 'flaccid paralysis'] 
Score: -10.974077224731445 	 ['covid-19' 'disease_symptom' 'emaciation'] 
Score: -11.037582397460938 	 ['covid-19' 'disease_symptom' 'illness'] 
Score: -11.059940338134766 	 ['covid-19' 'disease_symptom' 'febrile'] 
Score: -11.08718204498291 	 ['covid-19' 'disease_symptom' 'prodrome'] 
Score: -11.091679573059082 	 ['covid-19' 'd

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='symptom_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -9.79819393157959 	 ['finding' 'symptom_disease' 'covid-19'] 
Score: -9.902950286865234 	 ['severe diarrhea' 'symptom_disease' 'covid-19'] 
Score: -9.911456108093262 	 ['imbalance' 'symptom_disease' 'covid-19'] 
Score: -9.968975067138672 	 ['discharge' 'symptom_disease' 'covid-19'] 
Score: -10.052539825439453 	 ['symptom' 'symptom_disease' 'covid-19'] 
Score: -10.113763809204102 	 ['discomfort' 'symptom_disease' 'covid-19'] 
Score: -10.165021896362305 	 ['joint pain' 'symptom_disease' 'covid-19'] 
Score: -10.179061889648438 	 ['weakness' 'symptom_disease' 'covid-19'] 
Score: -10.205561637878418 	 ['cyanosis' 'symptom_disease' 'covid-19'] 
Score: -10.23859977722168 	 ['nasal congestion' 'symptom_disease' 'covid-19'] 
Score: -10.327827453613281 	 ['watery diarrhea' 'symptom_disease' 'covid-19'] 
Score: -10.34747314453125 	 ['syncope' 'symptom_disease' 'covid-19'] 
Score: -10.370210647583008 	 ['anesthesia' 'symptom_disease' 'covid-19'] 
Score: -10.378181457519531 	 ['pyrexia' 'sym

### Query 5: What are potential drugs to treat COVID-19?

In [ ]:

triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='drug_disease', 
                             tail='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -9.31729507446289 	 ['i' 'drug_disease' 'covid-19'] 
Score: -9.387167930603027 	 ['metformin' 'drug_disease' 'covid-19'] 
Score: -9.464078903198242 	 ['5' 'drug_disease' 'covid-19'] 
Score: -9.476957321166992 	 ['melatonin' 'drug_disease' 'covid-19'] 
Score: -9.489400863647461 	 ['ribavirin' 'drug_disease' 'covid-19'] 
Score: -9.519683837890625 	 ['vitamin d' 'drug_disease' 'covid-19'] 
Score: -9.534769058227539 	 ['glycyrrhizin' 'drug_disease' 'covid-19'] 
Score: -9.536579132080078 	 ['chloroquine' 'drug_disease' 'covid-19'] 
Score: -9.543299674987793 	 ['2' 'drug_disease' 'covid-19'] 
Score: -9.571304321289062 	 ['drug' 'drug_disease' 'covid-19'] 
Score: -9.615782737731934 	 ['β' 'drug_disease' 'covid-19'] 
Score: -9.617036819458008 	 ['curcumin' 'drug_disease' 'covid-19'] 
Score: -9.617403984069824 	 ['corticosteroid' 'drug_disease' 'covid-19'] 
Score: -9.6458740234375 	 ['3' 'drug_disease' 'covid-19'] 
Score: -9.656030654907227 	 ['arbidol' 'drug_disease' 'covid-19'] 


In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='disease_drug', 
                             head='covid-19', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -10.11438274383545 	 ['covid-19' 'disease_drug' '35'] 
Score: -10.127374649047852 	 ['covid-19' 'disease_drug' 'trizol'] 
Score: -10.203775405883789 	 ['covid-19' 'disease_drug' 'dmem'] 
Score: -10.26362419128418 	 ['covid-19' 'disease_drug' '3'] 
Score: -10.276357650756836 	 ['covid-19' 'disease_drug' '6'] 
Score: -10.295913696289062 	 ['covid-19' 'disease_drug' 'ci'] 
Score: -10.324422836303711 	 ['covid-19' 'disease_drug' '5'] 
Score: -10.331746101379395 	 ['covid-19' 'disease_drug' '2'] 
Score: -10.370031356811523 	 ['covid-19' 'disease_drug' 'pfu'] 
Score: -10.377532005310059 	 ['covid-19' 'disease_drug' 'fomite'] 
Score: -10.448404312133789 	 ['covid-19' 'disease_drug' 'met'] 
Score: -10.452817916870117 	 ['covid-19' 'disease_drug' 'α'] 
Score: -10.461031913757324 	 ['covid-19' 'disease_drug' 'od'] 
Score: -10.461099624633789 	 ['covid-19' 'disease_drug' 'dexamethasone'] 
Score: -10.4702730178833 	 ['covid-19' 'disease_drug' 'oxygen'] 


In [ ]:
# Compare the raw call and graph embedding results for the symptoms query

## raw call
#dataset.loc[(dataset['subject']=='symptom')].head(20)

#####==> the result of raw call seems to be better

###Query 7: What are treatment procedures related to treatment procedures for COVID-19?

###Using of 'diet','isolation', 'vaccination', 'coagulation' & 'stimulation'

###using 'diet'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_treat_procedure', 
                             tail='diet', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.080059051513672 	 ['diet' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.201196193695068 	 ['nephrectomy' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.300691604614258 	 ['aim' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.305400848388672 	 ['guidance' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.406518936157227 	 ['vaccination' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.409911155700684 	 ['delivery' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.4114837646484375 	 ['application' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.429228782653809 	 ['immunization' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.429618835449219 	 ['aspiration' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.443439483642578 	 ['tracheal intubation' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.4492902755737305 	 ['administration' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.449623107910156 	 ['oxygen therapy' 'treat_procedure

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_treat_procedure', 
                             head='diet', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.080059051513672 	 ['diet' 'treat_procedure_treat_procedure' 'diet'] 
Score: -7.711861610412598 	 ['diet' 'treat_procedure_treat_procedure' 'prevention'] 
Score: -7.819136142730713 	 ['diet' 'treat_procedure_treat_procedure' 'extubation'] 
Score: -7.828660488128662 	 ['diet' 'treat_procedure_treat_procedure' 'intervention'] 
Score: -7.881735801696777 	 ['diet' 'treat_procedure_treat_procedure' 'oral administration'] 
Score: -7.8910136222839355 	 ['diet' 'treat_procedure_treat_procedure' 'operation'] 
Score: -7.893488883972168 	 ['diet' 'treat_procedure_treat_procedure' 'procedure'] 
Score: -7.970276832580566 	 ['diet' 'treat_procedure_treat_procedure' 'glucocorticoid therapy'] 
Score: -7.971500396728516 	 ['diet' 'treat_procedure_treat_procedure' 'discussion'] 
Score: -7.972485542297363 	 ['diet' 'treat_procedure_treat_procedure' 'therapy'] 
Score: -7.976380825042725 	 ['diet' 'treat_procedure_treat_procedure' 'clearance'] 
Score: -7.987376689910889 	 ['diet' 'treat_procedure_

###using 'isolation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_treat_procedure', 
                             tail='isolation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.08005952835083 	 ['isolation' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.373478412628174 	 ['immunization' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.379122734069824 	 ['application' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.383002758026123 	 ['delivery' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.392162322998047 	 ['nephrectomy' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.4100494384765625 	 ['vaccination' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.451627731323242 	 ['guidance' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.485877990722656 	 ['change' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.5008087158203125 	 ['administration' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.502677917480469 	 ['reduction' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.505641937255859 	 ['aim' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.5338025

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_treat_procedure', 
                             head='isolation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.92832088470459 	 ['isolation' 'treat_procedure_treat_procedure' 'prevention'] 
Score: -7.08005952835083 	 ['isolation' 'treat_procedure_treat_procedure' 'isolation'] 
Score: -7.123196125030518 	 ['isolation' 'treat_procedure_treat_procedure' 'clearance'] 
Score: -7.15065336227417 	 ['isolation' 'treat_procedure_treat_procedure' 'intervention'] 
Score: -7.164436340332031 	 ['isolation' 'treat_procedure_treat_procedure' 'release'] 
Score: -7.173216819763184 	 ['isolation' 'treat_procedure_treat_procedure' 'extubation'] 
Score: -7.179913520812988 	 ['isolation' 'treat_procedure_treat_procedure' 'detection'] 
Score: -7.193852424621582 	 ['isolation' 'treat_procedure_treat_procedure' 'treatment'] 
Score: -7.194944381713867 	 ['isolation' 'treat_procedure_treat_procedure' 'reduction'] 
Score: -7.204606533050537 	 ['isolation' 'treat_procedure_treat_procedure' 'therapy'] 
Score: -7.2100043296813965 	 ['isolation' 'treat_procedure_treat_procedure' 'procedure'] 
Score: -7.218847274780

###using 'vaccination'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_treat_procedure', 
                             tail='vaccination', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.08005952835083 	 ['vaccination' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.308746337890625 	 ['immunization' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.345425605773926 	 ['delivery' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.438643455505371 	 ['application' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.494252681732178 	 ['administration' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.532453536987305 	 ['isolation' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.559513092041016 	 ['nephrectomy' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.568544387817383 	 ['guidance' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.586875915527344 	 ['reduction' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.590907096862793 	 ['change' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.615630149841309 	 ['presence' 'treat_procedure_treat_procedure' 'vaccina

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_treat_procedure', 
                             head='vaccination', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.838803291320801 	 ['vaccination' 'treat_procedure_treat_procedure' 'prevention'] 
Score: -7.042422294616699 	 ['vaccination' 'treat_procedure_treat_procedure' 'release'] 
Score: -7.08005952835083 	 ['vaccination' 'treat_procedure_treat_procedure' 'vaccination'] 
Score: -7.088526725769043 	 ['vaccination' 'treat_procedure_treat_procedure' 'intervention'] 
Score: -7.089054107666016 	 ['vaccination' 'treat_procedure_treat_procedure' 'clearance'] 
Score: -7.134792804718018 	 ['vaccination' 'treat_procedure_treat_procedure' 'treatment'] 
Score: -7.152895927429199 	 ['vaccination' 'treat_procedure_treat_procedure' 'detection'] 
Score: -7.1648406982421875 	 ['vaccination' 'treat_procedure_treat_procedure' 'reduction'] 
Score: -7.166986465454102 	 ['vaccination' 'treat_procedure_treat_procedure' 'induction'] 
Score: -7.169870376586914 	 ['vaccination' 'treat_procedure_treat_procedure' 'therapy'] 
Score: -7.188838958740234 	 ['vaccination' 'treat_procedure_treat_procedure' 'procedure'

###using 'coagulation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_treat_procedure', 
                             tail='coagulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.080059051513672 	 ['coagulation' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.155509948730469 	 ['nephrectomy' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.299044609069824 	 ['aim' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.316082000732422 	 ['guidance' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.336625576019287 	 ['delivery' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.338357925415039 	 ['aspiration' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.359601020812988 	 ['immunization' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.3657121658325195 	 ['vaccination' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.38093376159668 	 ['tracheal intubation' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.3866753578186035 	 ['application' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -7.449010848999023 	 ['oxygen therapy' 'treat_procedure_treat_proced

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_treat_procedure', 
                             head='coagulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.080059051513672 	 ['coagulation' 'treat_procedure_treat_procedure' 'coagulation'] 
Score: -8.002315521240234 	 ['coagulation' 'treat_procedure_treat_procedure' 'extubation'] 
Score: -8.081596374511719 	 ['coagulation' 'treat_procedure_treat_procedure' 'prevention'] 
Score: -8.189035415649414 	 ['coagulation' 'treat_procedure_treat_procedure' 'operation'] 
Score: -8.235719680786133 	 ['coagulation' 'treat_procedure_treat_procedure' 'procedure'] 
Score: -8.246740341186523 	 ['coagulation' 'treat_procedure_treat_procedure' 'intervention'] 
Score: -8.247796058654785 	 ['coagulation' 'treat_procedure_treat_procedure' 'release'] 
Score: -8.260361671447754 	 ['coagulation' 'treat_procedure_treat_procedure' 'destruction'] 
Score: -8.268036842346191 	 ['coagulation' 'treat_procedure_treat_procedure' 'prophylaxis'] 
Score: -8.287433624267578 	 ['coagulation' 'treat_procedure_treat_procedure' 'glucocorticoid therapy'] 
Score: -8.30639362335205 	 ['coagulation' 'treat_procedure_treat_pro

###using 'stimulation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='treat_procedure_treat_procedure', 
                             tail='stimulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.08005952835083 	 ['stimulation' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.323227405548096 	 ['immunization' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.327688217163086 	 ['vaccination' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.362116813659668 	 ['nephrectomy' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.397525310516357 	 ['change' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.404120922088623 	 ['delivery' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.4132890701293945 	 ['application' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.448145389556885 	 ['guidance' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.490756511688232 	 ['presence' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.510782718658447 	 ['reduction' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.5110321044921875 	 ['administration' 'treat_procedure_treat_procedure' 'sti

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_treat_procedure', 
                             head='stimulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.08005952835083 	 ['stimulation' 'treat_procedure_treat_procedure' 'stimulation'] 
Score: -7.207980155944824 	 ['stimulation' 'treat_procedure_treat_procedure' 'prevention'] 
Score: -7.294012069702148 	 ['stimulation' 'treat_procedure_treat_procedure' 'release'] 
Score: -7.403456211090088 	 ['stimulation' 'treat_procedure_treat_procedure' 'clearance'] 
Score: -7.428044319152832 	 ['stimulation' 'treat_procedure_treat_procedure' 'induction'] 
Score: -7.439124584197998 	 ['stimulation' 'treat_procedure_treat_procedure' 'intervention'] 
Score: -7.476195335388184 	 ['stimulation' 'treat_procedure_treat_procedure' 'detection'] 
Score: -7.478886604309082 	 ['stimulation' 'treat_procedure_treat_procedure' 'procedure'] 
Score: -7.48151969909668 	 ['stimulation' 'treat_procedure_treat_procedure' 'treatment'] 
Score: -7.489666938781738 	 ['stimulation' 'treat_procedure_treat_procedure' 'reduction'] 
Score: -7.5184783935546875 	 ['stimulation' 'treat_procedure_treat_procedure' 'discussio

###Query 8: What are treatment procedures related to genes for COVID-19?

###Using of 'diet','isolation', 'vaccination', 'coagulation' & 'stimulation'

###using 'diet'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='gene_treat_procedure', 
                             tail='diet', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.814241886138916 	 ['nucleic acid' 'gene_treat_procedure' 'diet'] 
Score: -7.823107719421387 	 ['step' 'gene_treat_procedure' 'diet'] 
Score: -7.9059624671936035 	 ['antibody' 'gene_treat_procedure' 'diet'] 
Score: -7.921473503112793 	 ['protein' 'gene_treat_procedure' 'diet'] 
Score: -7.92204475402832 	 ['mabs' 'gene_treat_procedure' 'diet'] 
Score: -7.942603588104248 	 ['viral antigen' 'gene_treat_procedure' 'diet'] 
Score: -7.98054838180542 	 ['mab' 'gene_treat_procedure' 'diet'] 
Score: -7.992313861846924 	 ['antigen' 'gene_treat_procedure' 'diet'] 
Score: -8.008453369140625 	 ['igg antibody' 'gene_treat_procedure' 'diet'] 
Score: -8.059837341308594 	 ['il' 'gene_treat_procedure' 'diet'] 
Score: -8.062214851379395 	 ['plasma' 'gene_treat_procedure' 'diet'] 
Score: -8.067042350769043 	 ['ifn' 'gene_treat_procedure' 'diet'] 
Score: -8.091503143310547 	 ['band' 'gene_treat_procedure' 'diet'] 
Score: -8.11323070526123 	 ['ifns' 'gene_treat_procedure' 'diet'] 
Score: -8.1135816

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_gene', 
                             head='diet', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.076888084411621 	 ['diet' 'treat_procedure_gene' 'step'] 
Score: -7.148949146270752 	 ['diet' 'treat_procedure_gene' 'serum'] 
Score: -7.191285133361816 	 ['diet' 'treat_procedure_gene' 'antigen'] 
Score: -7.259720802307129 	 ['diet' 'treat_procedure_gene' 'impact'] 
Score: -7.274569511413574 	 ['diet' 'treat_procedure_gene' 'antibody'] 
Score: -7.310152530670166 	 ['diet' 'treat_procedure_gene' 'ifn'] 
Score: -7.3261871337890625 	 ['diet' 'treat_procedure_gene' 'plasma'] 
Score: -7.332142353057861 	 ['diet' 'treat_procedure_gene' 'best'] 
Score: -7.346474647521973 	 ['diet' 'treat_procedure_gene' 'csf'] 
Score: -7.352807998657227 	 ['diet' 'treat_procedure_gene' 'protein'] 
Score: -7.408010959625244 	 ['diet' 'treat_procedure_gene' 'end'] 
Score: -7.418285369873047 	 ['diet' 'treat_procedure_gene' 'set'] 
Score: -7.43527889251709 	 ['diet' 'treat_procedure_gene' 'gene'] 
Score: -7.4656805992126465 	 ['diet' 'treat_procedure_gene' 'igg'] 
Score: -7.481998443603516 	 ['diet' '

###using 'isolation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='gene_treat_procedure', 
                             tail='isolation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.271273612976074 	 ['antibody' 'gene_treat_procedure' 'isolation'] 
Score: -7.279046058654785 	 ['step' 'gene_treat_procedure' 'isolation'] 
Score: -7.342713356018066 	 ['protein' 'gene_treat_procedure' 'isolation'] 
Score: -7.343426704406738 	 ['nucleic acid' 'gene_treat_procedure' 'isolation'] 
Score: -7.351099014282227 	 ['mabs' 'gene_treat_procedure' 'isolation'] 
Score: -7.385714530944824 	 ['viral antigen' 'gene_treat_procedure' 'isolation'] 
Score: -7.397027969360352 	 ['il' 'gene_treat_procedure' 'isolation'] 
Score: -7.3991193771362305 	 ['antigen' 'gene_treat_procedure' 'isolation'] 
Score: -7.428561210632324 	 ['igg antibody' 'gene_treat_procedure' 'isolation'] 
Score: -7.4393463134765625 	 ['mab' 'gene_treat_procedure' 'isolation'] 
Score: -7.449716567993164 	 ['plasma' 'gene_treat_procedure' 'isolation'] 
Score: -7.475085258483887 	 ['ifn' 'gene_treat_procedure' 'isolation'] 
Score: -7.5058488845825195 	 ['gene' 'gene_treat_procedure' 'isolation'] 
Score: -7.52886

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_gene', 
                             head='isolation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.358750343322754 	 ['isolation' 'treat_procedure_gene' 'serum'] 
Score: -6.413634777069092 	 ['isolation' 'treat_procedure_gene' 'step'] 
Score: -6.444091796875 	 ['isolation' 'treat_procedure_gene' 'antigen'] 
Score: -6.5059428215026855 	 ['isolation' 'treat_procedure_gene' 'plasma'] 
Score: -6.533306121826172 	 ['isolation' 'treat_procedure_gene' 'impact'] 
Score: -6.540275573730469 	 ['isolation' 'treat_procedure_gene' 'antibody'] 
Score: -6.553938865661621 	 ['isolation' 'treat_procedure_gene' 'csf'] 
Score: -6.565293312072754 	 ['isolation' 'treat_procedure_gene' 'ifn'] 
Score: -6.60703182220459 	 ['isolation' 'treat_procedure_gene' 'end'] 
Score: -6.61910343170166 	 ['isolation' 'treat_procedure_gene' 'set'] 
Score: -6.665585994720459 	 ['isolation' 'treat_procedure_gene' 'best'] 
Score: -6.677117824554443 	 ['isolation' 'treat_procedure_gene' 'protein'] 
Score: -6.727340221405029 	 ['isolation' 'treat_procedure_gene' 'interferon'] 
Score: -6.730746746063232 	 ['isolatio

###using 'vaccination'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='gene_treat_procedure', 
                             tail='vaccination', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.290629863739014 	 ['step' 'gene_treat_procedure' 'vaccination'] 
Score: -7.329212665557861 	 ['antibody' 'gene_treat_procedure' 'vaccination'] 
Score: -7.373039245605469 	 ['mabs' 'gene_treat_procedure' 'vaccination'] 
Score: -7.380733489990234 	 ['nucleic acid' 'gene_treat_procedure' 'vaccination'] 
Score: -7.393228530883789 	 ['protein' 'gene_treat_procedure' 'vaccination'] 
Score: -7.427067279815674 	 ['viral antigen' 'gene_treat_procedure' 'vaccination'] 
Score: -7.450476169586182 	 ['antigen' 'gene_treat_procedure' 'vaccination'] 
Score: -7.458548545837402 	 ['il' 'gene_treat_procedure' 'vaccination'] 
Score: -7.463231086730957 	 ['igg antibody' 'gene_treat_procedure' 'vaccination'] 
Score: -7.465262413024902 	 ['mab' 'gene_treat_procedure' 'vaccination'] 
Score: -7.499251365661621 	 ['ifn' 'gene_treat_procedure' 'vaccination'] 
Score: -7.537175178527832 	 ['plasma' 'gene_treat_procedure' 'vaccination'] 
Score: -7.551883220672607 	 ['gene' 'gene_treat_procedure' 'vaccina

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_gene', 
                             head='vaccination', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.197863578796387 	 ['vaccination' 'treat_procedure_gene' 'serum'] 
Score: -6.205445289611816 	 ['vaccination' 'treat_procedure_gene' 'step'] 
Score: -6.275914192199707 	 ['vaccination' 'treat_procedure_gene' 'antigen'] 
Score: -6.345757961273193 	 ['vaccination' 'treat_procedure_gene' 'antibody'] 
Score: -6.375162124633789 	 ['vaccination' 'treat_procedure_gene' 'plasma'] 
Score: -6.3784565925598145 	 ['vaccination' 'treat_procedure_gene' 'csf'] 
Score: -6.3867878913879395 	 ['vaccination' 'treat_procedure_gene' 'impact'] 
Score: -6.415197849273682 	 ['vaccination' 'treat_procedure_gene' 'ifn'] 
Score: -6.482891082763672 	 ['vaccination' 'treat_procedure_gene' 'end'] 
Score: -6.516984462738037 	 ['vaccination' 'treat_procedure_gene' 'best'] 
Score: -6.522115707397461 	 ['vaccination' 'treat_procedure_gene' 'protein'] 
Score: -6.531242370605469 	 ['vaccination' 'treat_procedure_gene' 'set'] 
Score: -6.546443939208984 	 ['vaccination' 'treat_procedure_gene' 'igg'] 
Score: -6.569

###using 'coagulation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='gene_treat_procedure', 
                             tail='coagulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -7.952884674072266 	 ['nucleic acid' 'gene_treat_procedure' 'coagulation'] 
Score: -7.970551490783691 	 ['step' 'gene_treat_procedure' 'coagulation'] 
Score: -8.024620056152344 	 ['mabs' 'gene_treat_procedure' 'coagulation'] 
Score: -8.031702995300293 	 ['antibody' 'gene_treat_procedure' 'coagulation'] 
Score: -8.033041000366211 	 ['protein' 'gene_treat_procedure' 'coagulation'] 
Score: -8.060356140136719 	 ['il' 'gene_treat_procedure' 'coagulation'] 
Score: -8.073432922363281 	 ['viral antigen' 'gene_treat_procedure' 'coagulation'] 
Score: -8.121862411499023 	 ['antigen' 'gene_treat_procedure' 'coagulation'] 
Score: -8.12790584564209 	 ['mab' 'gene_treat_procedure' 'coagulation'] 
Score: -8.136943817138672 	 ['igg antibody' 'gene_treat_procedure' 'coagulation'] 
Score: -8.164202690124512 	 ['plasma' 'gene_treat_procedure' 'coagulation'] 
Score: -8.177301406860352 	 ['she' 'gene_treat_procedure' 'coagulation'] 
Score: -8.184896469116211 	 ['gene' 'gene_treat_procedure' 'coagulat

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_gene', 
                             head='coagulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.948832988739014 	 ['coagulation' 'treat_procedure_gene' 'step'] 
Score: -7.003355026245117 	 ['coagulation' 'treat_procedure_gene' 'serum'] 
Score: -7.023144721984863 	 ['coagulation' 'treat_procedure_gene' 'antigen'] 
Score: -7.072268486022949 	 ['coagulation' 'treat_procedure_gene' 'plasma'] 
Score: -7.128332138061523 	 ['coagulation' 'treat_procedure_gene' 'ifn'] 
Score: -7.136817932128906 	 ['coagulation' 'treat_procedure_gene' 'impact'] 
Score: -7.14448356628418 	 ['coagulation' 'treat_procedure_gene' 'antibody'] 
Score: -7.150030136108398 	 ['coagulation' 'treat_procedure_gene' 'csf'] 
Score: -7.216067314147949 	 ['coagulation' 'treat_procedure_gene' 'end'] 
Score: -7.22769021987915 	 ['coagulation' 'treat_procedure_gene' 'set'] 
Score: -7.231782913208008 	 ['coagulation' 'treat_procedure_gene' 'protein'] 
Score: -7.23671817779541 	 ['coagulation' 'treat_procedure_gene' '12'] 
Score: -7.24821662902832 	 ['coagulation' 'treat_procedure_gene' 'interferon'] 
Score: -7.2505

###using 'stimulation'

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             head=None, 
                             relation='gene_treat_procedure', 
                             tail='stimulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -6.945173263549805 	 ['step' 'gene_treat_procedure' 'stimulation'] 
Score: -7.009547233581543 	 ['antibody' 'gene_treat_procedure' 'stimulation'] 
Score: -7.011922836303711 	 ['nucleic acid' 'gene_treat_procedure' 'stimulation'] 
Score: -7.042825698852539 	 ['protein' 'gene_treat_procedure' 'stimulation'] 
Score: -7.044106483459473 	 ['viral antigen' 'gene_treat_procedure' 'stimulation'] 
Score: -7.054532051086426 	 ['mabs' 'gene_treat_procedure' 'stimulation'] 
Score: -7.064726829528809 	 ['il' 'gene_treat_procedure' 'stimulation'] 
Score: -7.083166122436523 	 ['igg antibody' 'gene_treat_procedure' 'stimulation'] 
Score: -7.095091819763184 	 ['antigen' 'gene_treat_procedure' 'stimulation'] 
Score: -7.116360664367676 	 ['ifn' 'gene_treat_procedure' 'stimulation'] 
Score: -7.155575275421143 	 ['band' 'gene_treat_procedure' 'stimulation'] 
Score: -7.163001537322998 	 ['mab' 'gene_treat_procedure' 'stimulation'] 
Score: -7.178347110748291 	 ['ifns' 'gene_treat_procedure' 'stimulati

In [ ]:
triples, scores = query_topn(model, top_n=15, 
                             tail=None, 
                             relation='treat_procedure_gene', 
                             head='stimulation', 
                             ents_to_consider=None, 
                             rels_to_consider=None)

for triple, score in zip(triples, scores):
    print('Score: {} \t {} '.format(score, triple))

Score: -5.959142684936523 	 ['stimulation' 'treat_procedure_gene' 'serum'] 
Score: -5.986208915710449 	 ['stimulation' 'treat_procedure_gene' 'step'] 
Score: -6.067951202392578 	 ['stimulation' 'treat_procedure_gene' 'antigen'] 
Score: -6.1158447265625 	 ['stimulation' 'treat_procedure_gene' 'plasma'] 
Score: -6.124014377593994 	 ['stimulation' 'treat_procedure_gene' 'csf'] 
Score: -6.164010047912598 	 ['stimulation' 'treat_procedure_gene' 'ifn'] 
Score: -6.176165580749512 	 ['stimulation' 'treat_procedure_gene' 'impact'] 
Score: -6.1956987380981445 	 ['stimulation' 'treat_procedure_gene' 'antibody'] 
Score: -6.249466896057129 	 ['stimulation' 'treat_procedure_gene' 'best'] 
Score: -6.2654194831848145 	 ['stimulation' 'treat_procedure_gene' 'lp'] 
Score: -6.28163480758667 	 ['stimulation' 'treat_procedure_gene' 'end'] 
Score: -6.28341007232666 	 ['stimulation' 'treat_procedure_gene' 'set'] 
Score: -6.290707588195801 	 ['stimulation' 'treat_procedure_gene' 'interferon'] 
Score: -6.32033